# NiN块

NiN块使用 1x1 卷积层来替代全连接层，从而使空间信息能够自然传递到后面的层中去。

NiN块是NiN中的基础块。它由一个卷积层加两个充当全连接层的 1x1 卷积层串联而成。
其中第一个卷积层的超参数可以自行设定，而第二个和第三个卷积层的超参数一般是固定的

In [1]:
import sys
sys.path.append('../')

In [7]:
import mxnet as mx
import gluonbook as gb
from mxnet import gluon,init,nd
from mxnet.gluon import nn,data as gdata

In [3]:
def nin_block(num_channels,kernel_size,strides,padding):
    blk = nn.Sequential()
    blk.add(nn.Conv2D(num_channels,kernel_size,strides,padding,activation='relu'),
            nn.Conv2D(num_channels,kernel_size=1,activation='relu'),
            nn.Conv2D(num_channels,kernel_size=1,activation='relu')
           )
    return blk

In [4]:
nin_net = nn.Sequential()
nin_net.add(nin_block(96,kernel_size = 11,strides=4,padding=0),
            nn.MaxPool2D(pool_size=3,strides=2),
            nin_block(128,kernel_size =5,strides=1,padding=2),
            nn.MaxPool2D(pool_size=3,strides=2),
            nin_block(256,kernel_size =3,strides=1,padding=1),
            nn.MaxPool2D(pool_size=3,strides=2),nn.Dropout(0.5),
            
            #标签类别数为10
            nin_block(10,kernel_size=3,strides=1,padding=1),
            #全局平均池化层将窗口形状自动设置成输入的高和宽。
            nn.GlobalAvgPool2D(),
            #将输出变成二维的输出，其形状为（批量大小，10）
            nn.Flatten()
            )

In [5]:
X = nd.random.uniform(shape=(1,1,224,224),ctx=mx.gpu())
nin_net.initialize(ctx = mx.gpu())
for layer in nin_net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential1 output shape:	 (1, 96, 54, 54)
pool0 output shape:	 (1, 96, 26, 26)
sequential2 output shape:	 (1, 128, 26, 26)
pool1 output shape:	 (1, 128, 12, 12)
sequential3 output shape:	 (1, 256, 12, 12)
pool2 output shape:	 (1, 256, 5, 5)
dropout0 output shape:	 (1, 256, 5, 5)
sequential4 output shape:	 (1, 10, 5, 5)
pool3 output shape:	 (1, 10, 1, 1)
flatten0 output shape:	 (1, 10)


## 获取数据并进行训练

In [8]:
def load_data_fashion_mnist(batch_size,resize=None):
    transformer = []
    path = '../chapter1_baseKnowledge/FashionMNIST/'
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)
    mnist_train =gdata.vision.FashionMNIST(root=path,train=True)
    mnist_test =gdata.vision.FashionMNIST(root= path,train=False)
    
    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),batch_size,shuffle = True)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),batch_size,shuffle=False)
    
    return train_iter,test_iter

batch_size,lr,num_epochs ,ctx= 256,0.1,5,gb.try_gpu()
train_iter,test_iter = load_data_fashion_mnist(batch_size,resize=96)
trainer = gluon.Trainer(nin_net.collect_params(),'sgd',{'learning_rate':lr})
nin_net.initialize(force_reinit = True,init = init.Xavier(),ctx=ctx)

In [ ]:
gb.train_ch5(nin_net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on gpu(0)
epoch 1, loss 2.2898, train acc 0.141, test acc 0.195, time 29.2 sec
epoch 2, loss 1.6022, train acc 0.355, test acc 0.590, time 27.0 sec
